# Cmdkey Cached Credentials Recon
Detects usage of cmdkey to look for cached credentials

## Rule Content
```
- title: Cmdkey Cached Credentials Recon
  id: 07f8bdc2-c9b3-472a-9817-5a670b872f53
  status: experimental
  description: Detects usage of cmdkey to look for cached credentials
  references:
  - https://www.peew.pw/blog/2017/11/26/exploring-cmdkey-an-edge-case-for-privilege-escalation
  - https://technet.microsoft.com/en-us/library/cc754243(v=ws.11).aspx
  author: jmallette
  tags:
  - attack.credential_access
  - attack.t1003
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      Image: '*\cmdkey.exe'
      CommandLine: '* /list *'
    condition: selection
  fields:
  - CommandLine
  - ParentCommandLine
  - User
  falsepositives:
  - Legitimate administrative tasks.
  level: low

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_path.keyword:*\\cmdkey.exe AND process_command_line.keyword:*\ \/list\ *)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()